### Building a recommendation system with restaurants in London

### Ideas

#### < Maybe find a map dataset or something to get geo data for each restaurant, would be nice to have a cuisine datapoint too

#### < NLP angle with the reviews textual data

#### < Can use date as a factor, more recent the review, the more weighting it should have

#### < What does good look like? Like what is a 'good' recommendation?

_______________________________________________________________________

### Factor Release Plan

#### Release 1

##### 1. Initial recommender based on review value and volume
##### 2. Involve date as a factor of review reliability
##### 3. Introduce textual NLP analysis of reviews as another way of weighting
##### 4. Bring in geo data for people to select areas or something?
##### 5. Would be nice to webscrape cuisine as a datapoint too
##### 6. Could also include author id as a factor - potentially someone who posts lots of reviews are more reliable?

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer
from skorch.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline
from skorch import NeuralNetRegressor
import pickle
import emoji
import requests
from bs4 import BeautifulSoup

In [2]:
initial_df = pd.read_csv('Data/London_reviews.csv')

C:\Users\ndsna\AppData\Local\Temp\ipykernel_16148\2783312507.py:1: DtypeWarning: Columns (0,1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  initial_df = pd.read_csv('Data/London_reviews.csv')


In [3]:
print(initial_df)

       Unnamed: 0 parse_count       restaurant_name rating_review    sample  \
0               0           1  Cocotte_Notting_Hill           5.0  Positive   
1               1           2  Cocotte_Notting_Hill           5.0  Positive   
2               2           3  Cocotte_Notting_Hill           5.0  Positive   
3               3           4  Cocotte_Notting_Hill           5.0  Positive   
4               4           5  Cocotte_Notting_Hill           5.0  Positive   
...           ...         ...                   ...           ...       ...   
996562     999995      999996       The_Old_Brewery           4.0  Positive   
996563     999996      999997       The_Old_Brewery           2.0  Negative   
996564     999997      999998       The_Old_Brewery           5.0  Positive   
996565     999998      999999       The_Old_Brewery           5.0  Positive   
996566     999999     1000000       The_Old_Brewery           3.0  Negative   

               review_id                           

In [4]:
df = initial_df.drop(['Unnamed: 0','parse_count'],axis=1)

In [ ]:
# testing scraping geo data from Google Maps

url = "https://www.google.com/maps/search/Cocotte%20Notting%20Hill"
response = requests.get(url)
html_content = response.text

soup = BeautifulSoup(html_content, "html.parser")

print(html_content)
# restaurants = soup.find_all("div", class_="section-result-details-container")

# for restaurant in restaurants:
#     name = restaurant.find("h3", class_="section-result-title").text.strip()
#     address = restaurant.find("span", class_="section-result-location").text.strip()
    
#     print("Name:", name)
#     print("Address:", address)
#     print("-" * 50)

In [15]:
# Import the library Selenium 
from selenium import webdriver 
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
  
# Make browser open in background 
options = webdriver.ChromeOptions() 
#options.add_argument('headless') 
  
# Create the webdriver object 
browser = webdriver.Chrome(options=options) 
  
# Obtain the Google Map URL 
url = ["https://www.google.com/maps/search/Cocotte%20Notting%20Hill","https://www.google.com/maps/search/The%20Old%20Brewery"] 
# Initialize variables and declare it 0 
i = 0

results = []
# Create a loop for obtaining data from URLs 
for i in range(len(url)): 
    print(i)
    print(url[i])
    # Open the Google Map URL 
    browser.get(url[i]) 
    
    if i == 0:
        browser.find_element(By.XPATH,"//button[@aria-label='Accept all']").click()
        
        title = browser.find_element(By.XPATH, "//h1[@class='DUwDvf lfPIob']")
        print(i+1, "-", title.text) 
        
    if i == 1:    
        elems = browser.find_element(By.CSS_SELECTOR,("a.hfpxzc"))
        print(elems.text)
        # elems = browser.find_elements(By.XPATH, "//div[@role='article']") # "//a[@class='hfpxzc']"
        # for elem in elems:
            # print(elem.text)
            # title = elem.find_element(By.CSS_SELECTOR, "div.fontHeadlineSmall")
            # description = elem.find_element(By.CSS_SELECTOR, "div.fontBodyMedium")
            # results.append(str(title.text)+';'+str(description.text))
            # print(results)
    # Obtain the title of that place 
    # title = browser.find_element_by_class_Nname( 
    #     "x3AX1-LfntMc-header-title-title") 
    # title = browser.find_element(By.XPATH, "//span[@class='a5H0ec']")

  
    # Obtain the ratings of that place 
    # stars = browser.find_element_by_class_name("aMPvhf-fI6EEc-KVuj8d") 
    # print("The stars of restaurant are:", stars.text) 
    # print("\n") 
    
browser.quit()

0
https://www.google.com/maps/search/Cocotte%20Notting%20Hill
1 - Cocotte Notting Hill
1
https://www.google.com/maps/search/The%20Old%20Brewery



In [10]:
browser.get('https://www.google.com/maps/search/Cocotte%20Notting%20Hill')

In [ ]:
print(title.text)

In [13]:
for elem in elems:
    print(elem.)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.225)
Stacktrace:
	GetHandleVerifier [0x00007FF73DA72142+3514994]
	(No symbol) [0x00007FF73D690CE2]
	(No symbol) [0x00007FF73D5376AA]
	(No symbol) [0x00007FF73D510AFD]
	(No symbol) [0x00007FF73D5ACB1B]
	(No symbol) [0x00007FF73D5C218F]
	(No symbol) [0x00007FF73D5A5D93]
	(No symbol) [0x00007FF73D574BDC]
	(No symbol) [0x00007FF73D575C64]
	GetHandleVerifier [0x00007FF73DA9E16B+3695259]
	GetHandleVerifier [0x00007FF73DAF6737+4057191]
	GetHandleVerifier [0x00007FF73DAEE4E3+4023827]
	GetHandleVerifier [0x00007FF73D7C04F9+689705]
	(No symbol) [0x00007FF73D69C048]
	(No symbol) [0x00007FF73D698044]
	(No symbol) [0x00007FF73D6981C9]
	(No symbol) [0x00007FF73D6888C4]
	BaseThreadInitThunk [0x00007FFBE96F257D+29]
	RtlUserThreadStart [0x00007FFBEAC8AA58+40]


<selenium.webdriver.chrome.webdriver.WebDriver (session="8d45435ccfa366d5cb0b518f84d58b88")>


In [ ]:
browser.quit()

In [18]:
import requests
import json
from pprint import pprint

url = ["https://www.google.com/maps/search/Cocotte%20Notting%20Hill","https://www.google.com/maps/search/The%20Old%20Brewery"] 
r=requests.get("https://www.google.com/maps/search/Cocotte%20Notting%20Hill")

txt = r.text

find1 = "window.APP_INITIALIZATION_STATE="
find2 = ";window.APP"

i1 = txt.find(find1)
i2 = txt.find(find2, i1+1 )
js = txt[i1+len(find1):i2]
data = json.loads(js)
pprint(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)